In [15]:
import matplotlib.pyplot as plt
import os
import random 
import cv2
import numpy as np
import requests
import warnings
import google.generativeai as genai
import firebase_admin
import logging
import tempfile
import dotenv

dotenv.load_dotenv()

from urllib.parse import urlparse
from firebase_admin import credentials, firestore, storage
from ultralytics import YOLO
from gradio_client import Client

random.seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [16]:
cred = credentials.Certificate("credential/credential.json")
firebase_admin.initialize_app(cred) if not firebase_admin._apps else firebase_admin.get_app()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [17]:
db = firestore.client()
bucket = storage.bucket(name='planktosee-temp-file')
logger.info("Firebase initialized successfully.")

INFO:__main__:Firebase initialized successfully.


In [120]:
try:
    db = firestore.client()
    bucket = storage.bucket(name='planktosee-temp-file')
    logger.info("Connected to Firestore successfully.")
    
except Exception as e:
    logger.error(f"Error connecting to Firestore: {e}")

INFO:__main__:Connected to Firestore successfully.


In [ ]:
def extract_gcs_path(image_url):
    parsed_url = urlparse(image_url)
    path_parts = parsed_url.path.lstrip("/").split("/", 1)
    
    if len(path_parts) > 1:
        return path_parts[1] 
    return None  

def load_image_from_firebase(image_url):
    gcs_path = extract_gcs_path(image_url)
    if not gcs_path:
        raise ValueError(f"Invalid GCS URL: {image_url}")

    blob = bucket.blob(gcs_path)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        temp_file_path = temp_file.name 

    blob.download_to_filename(temp_file_path)
    
    return temp_file_path 

In [ ]:
import tempfile

def load_image_from_firebase(image_url):
    blob = bucket.blob(image_url)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        blob.download_to_filename(temp_file.name)
        temp_file_path = temp_file.name 

    return temp_file_path 

In [ ]:
def load_text_from_firebase(text_url):
    response = requests.get(text_url)
    
    if response.status_code == 200:
        text = response.text
    else:
        logger.error(f"Failed to load text from Firebase: {response.status_code}")
    return text

In [ ]:
def upload_image_to_firebase(image_path):
    blob = bucket.blob('images/' + os.path.basename(image_path))
    blob.upload_from_filename(image_path)
    blob.make_public()
    return blob.public_url

'https://storage.googleapis.com/planktosee-temp-file/images/signature.png'

In [ ]:
def remove_image_from_firebase(image_path):
    blob = bucket.blob('images/' + os.path.basename(image_path))
    blob.delete()
    logger.info(f"Image {image_path} removed from Firebase Storage.")

INFO:__main__:Image signature.png removed from Firebase Storage.


In [ ]:
models = genai.list_models()
for model in models:
    print(model.name)

In [ ]:
def gemini(message):
    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

    chatbot = genai.GenerativeModel("gemini-1.5-flash-002")
    system = f"""
        Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
        1. *Hierarki Taksonomi dalam Plankton:*
        2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
        3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
        4. *Klasifikasi Berdasarkan Habitat:*  
        5. *Klasifikasi Berdasarkan Siklus Hidup:*  
        6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
        7. *Klasifikasi Berdasarkan Ukuran:*  
        """
    response = chatbot.generate_content(system)
    return response.text

def qwen2(message):
    client = Client("Qwen/Qwen2-57b-a14b-instruct-demo")
    result = client.predict(
        query=message,
        history=[],
        system=
            f"""
            Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
            1. *Hierarki Taksonomi dalam Plankton:*
            2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
            3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
            4. *Klasifikasi Berdasarkan Habitat:*  
            5. *Klasifikasi Berdasarkan Siklus Hidup:*  
            6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
            7. *Klasifikasi Berdasarkan Ukuran:*  
            """,
        api_name="/model_chat"
    )
    return result[1][0][1]

def deepseek(message):
    client = Client("Abubekersiraj/Deepseek")
    result = client.predict(
        message=message,
        system_message=f"""
            Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
            1. *Hierarki Taksonomi dalam Plankton:*
            2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
            3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
            4. *Klasifikasi Berdasarkan Habitat:*  
            5. *Klasifikasi Berdasarkan Siklus Hidup:*  
            6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
            7. *Klasifikasi Berdasarkan Ukuran:*  
            """,
        max_tokens=2048,
        temperature=0.1,
        top_p=0.95,
        api_name="/chat"
    )
    return result

deepseek("Plankton")

In [ ]:
def predict_img(model_option, llm_option, img_path):
    if model_option == "yolov8-detect":
        model = YOLO("../model/yolov8-detect.pt")
    elif model_option == "yolov8-acvit":
        model = YOLO("../model/yolov8-acvit.pt")
    else:
        return "Model tidak ditemukan"
    
    img = cv2.imread(img_path)
    img = cv2.resize(img, (864, 576))

    results = model(img)

    cv2.imwrite(
        filename="static/uploads/detect_img.jpg", 
        img=cv2.cvtColor(results[0].plot(), cv2.COLOR_BGR2RGB)
    )

    detected_classes = [model.names[int(box.cls)] for box in results[0].boxes]
    confidences = [float(box.conf) for box in results[0].boxes]

    if llm_option == "qwen":
        response = qwen2(detected_classes)
    elif llm_option == "deepseek":
        response = deepseek(detected_classes)
    else:
        response = "Pilih model LLM yang sesuai."

    return detected_classes, confidences, response

In [5]:
detected_classes, confidences, response = predict_img("yolov8-detect", "qwen", "noctilluca_scintillans.jpg")


0: 448x640 1 noctilluca_scintillans, 249.4ms
Speed: 12.1ms preprocess, 249.4ms inference, 8.2ms postprocess per image at shape (1, 3, 448, 640)
Loaded as API: https://qwen-qwen2-57b-a14b-instruct-demo.hf.space ✔


In [2]:
import requests

text_link = "https://storage.googleapis.com/planktosee-temp-file/texts/f7e383ebfe1f46ad8e33638527a4abd5.txt"

response = requests.get(text_link)
if response.status_code == 200:
    text = response.text
else:
    logger.error(f"Failed to load text from Firebase: {response.status_code}")

print(text)

## Taksonomi dan Klasifikasi Plankton, dengan Fokus pada *Dinophysis miles*

*Dinophysis miles* adalah spesies dinoflagellata, sejenis plankton yang bersifat mikroskopis.  Untuk memahami klasifikasinya secara mendalam, mari kita bahas aspek-aspek yang Anda minta:

**1. Hierarki Taksonomi dalam Plankton:**

Sistem taksonomi modern mengikuti hierarki Linnaean, yang mengelompokkan organisme berdasarkan kesamaan karakteristik. Untuk *Dinophysis miles*, hierarkinya kira-kira seperti ini:

* **Domain:** Eukarya (organisme dengan sel eukariotik)
* **Kerajaan:** Protista (organisme eukariotik yang tidak termasuk hewan, tumbuhan, atau jamur)
* **Filum:** Dinoflagellata (organisme bersel tunggal dengan dua flagela)
* **Kelas:** Dinophyceae (sebagian besar dinoflagellata)
* **Ordo:** Dinophysales
* **Famili:** Dinophysiaceae
* **Genus:** *Dinophysis*
* **Spesies:** *Dinophysis miles*

Perhatikan bahwa hierarki ini bisa bervariasi sedikit tergantung pada sistem klasifikasi yang digunakan dan penem

In [1]:
import requests

url = "http://127.0.0.1:8000/result"

file = "sample_image.jpg"
model_option = "yolov8-acvit"
llm_option = "qwen"

with open(file, "rb") as f:
    files = {"img_path": f}
    
    data = {
        "model_option": model_option,
        "llm_option": llm_option
    }

    response = requests.post(url, files=files, data=data)

response.json()

{'img_path': 'https://storage.googleapis.com/planktosee-temp-file/images/1d736594507c4915a174fe8e642b23b0.jpg',
 'predictions': [['Noctilluca Scintillans', '0.964755']],
 'response': 'https://storage.googleapis.com/planktosee-temp-file/texts/79edf46e99ee468fa91d9bcbe0529d52.txt'}